In [ ]:
import os
import openai

from qdrant_client import QdrantClient
from langsmith import Client

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

## Get reference data from langsmith

In [33]:
client = Client(api_key=os.getenv("LANGCHAIN_API_KEY"))

In [34]:
dataset = client.read_dataset(dataset_name="rag-evaluation-dataset")

In [35]:
list_examples = list(client.list_examples(dataset_id=dataset.id, limit=10))
reference_input = list_examples[8].inputs
reference_output = list_examples[8].outputs

In [36]:
reference_input

{'question': "Are there any men's sneakers suitable for casual wear in your store?"}

In [37]:
reference_output

{'ground_truth': "Our men's sneaker options include the POLO RALPH LAUREN Train 90 Sneaker and Bruno Marc Fashion Sneakers, both designed for comfort and style.",
 'reference_context_ids': ['B0C649RHHN', 'B09QM776N9'],
 'reference_descriptions': ["POLO RALPH LAUREN Men's Train 90 Sneaker Imported Rubber sole Shaft measures approximately low-top from arch Rounded toe Lace-up front Inspired by classic running shoes from The late 1970s, the train 90 sneaker is designed with a low profile and an iconic percent-wing patch from polo’s stadium collection. I t e m   m o d e l   n u m b e r :   8 0 9 7 3 6 2 5 8 0 0 6 .   D e p a r t m e n t :   m e n s .   D a t e   F i r s t   A v a i l a b l e :   M a y   2 4 ,   2 0 2 3 .   M a n u f a c t u r e r :   P o l o   R a l p h   L a u r e n Clothing, Shoes & Jewelry Men Shoes Fashion Sneakers",
  "Bruno Marc Men's Fashion Sneakers Casual Comfort Canvas Skate Shoes 100% Canvas Imported Rubber sole Premium canvas upper is comfortable and soft to th

## RAG Pipeline

In [38]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [39]:

def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_images = []
    retrieved_metadata = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        similarity_scores.append(result.score)
        
        # Extract image URL if available
        image_url = result.payload.get("image", None)
        retrieved_images.append(image_url)
        
        # Extract other metadata
        metadata = {
            "parent_asin": result.payload.get("parent_asin"),
            "price": result.payload.get("price"),
            "average_rating": result.payload.get("average_rating"),
            "rating_number": result.payload.get("rating_number"),
            "store": result.payload.get("store"),
            "video": result.payload.get("video"),
            "image": result.payload.get("image")
        }
        retrieved_metadata.append(metadata)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similarity_scores": similarity_scores,
        "retrieved_images": retrieved_images,
        "retrieved_metadata": retrieved_metadata,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk in zip(context["retrieved_context_ids"], context["retrieved_context"]):
        formatted_context += f"- {id}: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
    )

    return response.choices[0].message.content


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    # Filter out None values from images and create a list of valid image URLs
    valid_images = [img for img in retrieved_context["retrieved_images"] if img is not None]
    
    # Filter metadata to only include items with images
    valid_metadata = [meta for meta, img in zip(retrieved_context["retrieved_metadata"], retrieved_context["retrieved_images"]) if img is not None]

    final_result = {
        "answer": answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"],
        "images": valid_images,
        "product_metadata": valid_metadata
    }

    return final_result

In [40]:
result = rag_pipeline(reference_input["question"], qdrant_client)

In [41]:
result

{'answer': "Yes, there are men's sneakers suitable for casual wear available. For example, the Bruno Marc Men's Fashion Sneakers (B09QM776N9) are casual comfort canvas skate shoes made with premium canvas upper and memory foam insole for cushioning and all-day comfort. Additionally, the POLO RALPH LAUREN Men's Train 90 Sneaker (B0C649RHHN) is a stylish low-top sneaker inspired by classic running shoes, suitable for casual wear.",
 'question': "Are there any men's sneakers suitable for casual wear in your store?",
 'retrieved_context_ids': ['B09QM776N9',
  'B0BBVS49J9',
  'B0C649RHHN',
  'B09S8J4QC5',
  'B09VN35T1M'],
 'retrieved_context': ["Bruno Marc Men's Fashion Sneakers Casual Comfort Canvas Skate Shoes 100% Canvas Imported Rubber sole Premium canvas upper is comfortable and soft to the touch and also provide advanced upper stitching workmanship make the men shoes durable enough to last a long time Soft inner material can provide a comfortable foot environment and prevent feet from

## RAGAS Metrics

In [84]:
from ragas.dataset_schema import SingleTurnSample 
from ragas.metrics import LLMContextPrecisionWithReference, LLMContextRecall, Faithfulness, AnswerRelevancy

In [45]:
ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

In [49]:
async def ragas_faithfulness(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = Faithfulness(llm=ragas_llm)

    return await scorer.single_turn_ascore(sample)

In [50]:
await ragas_faithfulness(result, "")

0.625

In [64]:
async def ragas_response_relevancy (run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = AnswerRelevancy(llm=ragas_llm, embeddings = ragas_embeddings)

    return await scorer.single_turn_ascore(sample)

In [65]:
await ragas_response_relevancy(result, "")

np.float64(0.8825538245737362)

In [87]:
async def ragas_context_precision(run, example):

    sample = SingleTurnSample(
        user_input=run["question"],
        reference=example["reference_descriptions"],
        retrieved_contexts=run["retrieved_context"]
    )
    scorer = LLMContextPrecisionWithReference(llm=ragas_llm)

    return await scorer.single_turn_ascore(sample)

In [88]:
await ragas_context_precision(result, reference_output)

ValidationError: 1 validation error for SingleTurnSample
reference
  Input should be a valid string [type=string_type, input_value=["POLO RALPH LAUREN Men's...Shoes Fashion Sneakers"], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type